### Use Case 2.3: What differences are there across datasets in volume and elevation curves of a reservoir? 

This notebook demonstrates basic WaMDaM use cases analysis using scientific Python libraries such as [pandas](https://pandas.pydata.org/) and [plotly](https://plot.ly/).  It reads WaMDaM SQLite data from a published HydroShare Generic Resource, runs SQL script, and them uses Python plotly to visualize the results

This use case identifies five volume-elevation curves for Hyrum Reservoir, Utah from three datasets: USBOR, Utah Dams, and WEAP model datasets

For more info: http://docs.wamdam.org/UseCases/use_case_2/#use-case-2.3



In [2]:

import os
from utilities import hydroshare
#from hs_restclient import HydroShare

import sqlite3
import numpy as np
import pandas as pd




In [3]:
!pip install plotly  # python 2.7
!pip3 install plotly # python3
!conda install -c plotly plotly -y


    100% |████████████████████████████████| 25.0MB 16kB/s  eta 0:00:01
  Running setup.py bdist_wheel for plotly ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/0c/3e/07/4848195c61f659184ca41d5a614845a018ab2d2f2a705b9998
Successfully built plotly
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
/bin/sh: 1: pip3: not found
Fetching package metadata ...................
Solving package specifications: .

Package plan for installation in environment /opt/conda:

The following NEW packages will be INSTALLED:

    plotly:       2.7.0-py_1                    conda-forge

The following packages will be UPDATED:

    scikit-learn: 0.18.2-py36_blas_openblas_201 conda-forge [blas_openblas] --> 0.19.1-py36_nomklh27f7947_0 defaults [nomkl]
    scipy:        0.19.1-py36_blas_openblas_202 conda-forge [blas_openblas] --> 1.1.0-py36_nomklh9d22d0a_0  defaults [nomkl]

The following packages wil


WaMDaM database test file (SQLite) on HydroShare
https://www.hydroshare.org/resource/1601e9f029984a87affcd94af6b4bad0/



import os
from utilities import hydroshare
#from hs_restclient import HydroShare

import sqlite3
import numpy as np
import pandas as pd

Next we need to establish a secure connection with HydroShare. This is done by simply instantiating the hydroshare class that is defined within hs_utils. In addition to connecting with HydroShare, this command also sets environment variables for several parameters that may useful to you:

Your username
The ID of the resource which launched the notebook
The type of resource that launched this notebook
The url for the notebook server.

In [ ]:
# establish a secure connection to HydroShare
hs = hydroshare.hydroshare()

In [ ]:
### Retrieve a resource using its ID

# The data for our processing routines can be retrieved using the `getResourceFromHydroShare` function by passing it the global identifier from the url above
# get some resource content. The resource content is returned as a dictionary
# Abdallah, A. (2018). Bear River Datasets, HydroShare, http://www.hydroshare.org/resource/bec9b20118804d119c4bfc232caea559
content = hs.getResourceFromHydroShare('bec9b20118804d119c4bfc232caea559')



In [ ]:
# Use SQLite engine to connect to the SQLite file. 
# Then we can run queries against it within this notebook :)  
conn = sqlite3.connect(hs.content["BearRiverDatasets_Jan2018.sqlite"])


In [ ]:
# Query multiple volume/elevation curves into one table to plot them together

# 4.2MultiAttributeValues.csv
import urllib

# for simplicity, we call the query directly from GitHub as a text. You may use the query here but it will be too long to show in a cell 
#https://github.com/WamdamProject/WaMDaM_UseCases/blob/master/UseCases_files/4Queries_SQL/UseCase2/UseCase2.3/4_MultiAttributeValues.sql

txt1 = urllib.urlopen("https://raw.githubusercontent.com/WamdamProject/WaMDaM_UseCases/master/UseCases_files/4Queries_SQL/UseCase2/UseCase2.3/3_MultipleTimeSeriesColumnsSameTimeStamp.sql").read()

# pass the query to the SQLite connection
df = pd.read_sql_query(txt1, conn)

# Save the query result into a CSV file
df.to_csv('query_resut.csv')

# to print the table result here within the notebook, just uncomment the df line below
#df


In [ ]:
# Get two time series with the same time stamp and merge them to be ready to plot them as part of the Volume-Elevation curve

# 4.3MergeTimeSeriesValues.sql
import urllib
txt2 = urllib.urlopen("https://raw.githubusercontent.com/WamdamProject/WaMDaM_UseCases/master/UseCases_files/4Queries_SQL/UseCase2/UseCase2.3/3_MergeTimeSeriesValues.sql").read()

# pass the query to the SQLite connection
df2 = pd.read_sql_query(txt2, conn)

# Save the query result into a CSV file
df2.to_csv('query_resut2.csv')



In [ ]:
# Import plotly libraries and set it to the notebook mode to embed the figures within a cell
import plotly.offline as offline
import plotly.graph_objs as go

offline.init_notebook_mode()

In [ ]:
# UseCase2.3_HyrumReservoir_Curves.py

# plot multi-attributes from multiple sources


# Adel Abdallah
# Jan 25, 2018

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from random import randint
import pandas as pd

## read the input data from GitHub csv file which is a direct query output for these queries:

# 4.2MultiAttributeValues.csv
df = pd.read_csv("https://raw.githubusercontent.com/WamdamProject/WaMDaM_UseCases/master/UseCases_files/5Results_CSV/4.2MultiAttributeValues.csv")

# 4.3MergeTimeSeriesValues.sql
df2 = pd.read_csv("https://raw.githubusercontent.com/WamdamProject/WaMDaM_UseCases/master/UseCases_files/5Results_CSV/4.3MergeTimeSeriesValues.csv")


subsets = df.groupby('ScenarioName')
data = []

#for each subset (curve), set up its legend and line info manually so they can be edited
subsets_settings = {
    'Utah Dams shapefile_as is': { # this oone is the name of subset as it appears in the csv file
        'dash': 'solid',     # this is properity of the line (curve)
        'width':'3',
        'legend_index': 1,   # to order the legend
         'symbol':'square',
        'size':'7',
        'mode':'line+markers',
        'legend_name': 'Utah Dams Dataset (2016)',  # this is the manual curve name 
         'color':'#990F0F'
        },
    
    'USU WEAP Model 2017': {
        'dash': 'solid',
         'width':'3',
          'mode':'line+markers',
          'symbol':'circle',
                'size':'7',

        'legend_index': 3,
        'legend_name': 'USU WEAP Model (2017)',
         'color':'#B26F2C'
        },
    'UDWR GenRes 2010': {
        'dash': 'dash',
        'mode':'line+markers',
        'width':'3',
                'size':'7',

        'symbol':'circle',
        'legend_index': 4,
        'legend_name': 'UDWR WEAP Model (2010)',
         'color':'#7A430C'
        },
    'Rwise': {
        'dash': 'dash',
        'mode':'line+markers',
        'width':'3',
                  'symbol':'star',
                'size':'7',

        'legend_index': 0,
        'legend_name': 'BOR Water Info. System (2017)',
         'color':'#E57E7E'
        },
    'Base case': {
        'dash': 'solid',
        'mode':'lines+markers',
        'width':'3',
                  'symbol':'bowtie',
        'size':'11',

        'legend_index': 2,
        'legend_name': 'BOR Reservoirs Dataset (2006)',
         'color':'#E5B17E'
        },    

    }


# This dict is used to map legend_name to original subset name
subsets_names = {y['legend_name']: x for x,y in subsets_settings.iteritems()}

      
#for each subset (curve), set up its legend and line info manually so they can be edited
subsets_settings2 = {
        'dash': 'solid',     # this is properity of the line (curve)
        'legend_index': 3,   # to order the legend
         'mode':'lines+markers',
        'color':'#E57E7E',
        'legend_name': 'BOR Water Info. System (2017)'  # this is the edited curve name 
                    }


# Get data from first dataframe (Multi-Attributes)
for subset in subsets.groups.keys():
    print subset
    scenario_name_data = subsets.get_group(name=subset)
    subsets_of_scenario = scenario_name_data.groupby("AttributeNameCV")
    s = go.Scatter(
                    x=subsets_of_scenario.get_group(name='Volume').Value,
                    y=subsets_of_scenario.get_group(name='Elevation').Value,
                        mode='lines+markers',

                    name = subsets_settings[subset]['legend_name'],
                    line = dict(
                        color =subsets_settings[subset]['color'],
                        width =subsets_settings[subset]['width'],
                        dash=subsets_settings[subset]['dash']
                                ),
                     marker = dict(
                         size=subsets_settings[subset]['size'],
                         symbol=subsets_settings[subset]['symbol'],
                         #color = '#a50021',
                         maxdisplayed=12
),  
                    opacity = 1)
    data.append(s)




# Get data from second dataframe (merged two time series as two Multi-Attributes)
data2 = go.Scatter(
                x=df2.VolumeValue,
                y=df2['ElevationValue'],
                name = subsets_settings2['legend_name'],
                mode='lines+markers',
                line = dict(
                    color ='#E57E7E',
                    width ='3'),
                marker = dict(
                size ='9',
                color = '#E57E7E',
                maxdisplayed=20,
                symbol ='star',
                         line = dict(
                         color = ['rgb(153, 84, 15)']
                         ),

                            ),
    
    
                opacity =1)
                
data.append(data2)     
    
# Legend is ordered based on data, so we are sorting the data based 
# on desired legend order indicarted by the index value entered above
data.sort(key=lambda x: subsets_settings[subsets_names[x['name']]]['legend_index'])


trace1 = go.Scatter(
    x=[1500, 8000, 16000],
    y=[4680, 4680,4680],
    mode='text',
    showlegend=False,
    text=['Dead<br> storage', 'Live<br>storage', 'Total<br>storage'],
    textposition='top',

)
data.append(trace1)     


# set up the figure layout
layout = {
        'shapes': [
        # Rectangle reference to the axes
        {
            "opacity": 0.3,
            'type': 'rect',
            'xref': 'x',
            'yref': 'y',
            'x0': 0,
            'y0': 4580,
            'x1': 3012,
            'y1': 4750,
            'line': {
                'color': 'rgb(0, 0, 0)',
                'width': 0.1,
            },
            'fillcolor': 'rgb(153, 229, 255)'
        },
     # Rectangle reference to the plot
        {
           "opacity": 0.3,
            'type': 'rect',
            'xref': 'x',
            'yref': 'y',
            'x0': 3012,
            'y0': 4580,
            'x1': 14440,
            'y1': 4750,
            'line': {
                'color': 'rgb(0, 0, 0)',
                'width': 0.1,
            },
            'fillcolor': 'rgb(127, 212, 255)',
        },
        
        {
            "opacity": 0.3,
            'type': 'rect',
            'xref': 'x',
            'yref': 'y',
            'x0': 14440,
            'y0': 4580,
            'x1': 17746,
            'y1': 4750,
            'line': {
                'color': 'rgb(0, 0, 0)',
                'width': 0.1,
            },
            'fillcolor': 'rgb(101, 191, 255)',
        }        
    ],
        'yaxis': {
        'title': 'Elevation (feet)',
        'tickformat': ',',
        'ticks':'outside',
        'ticklen': '10',


        'range' : ['4580', '4700'],
                'showline':'True'

                },
    'xaxis' : {
        'title' : 'Volume (acre-feet)',
        'tickformat': ',',   
         'showgrid':False,

        'ticks':'outside',
        'dtick':'5000',
        'range' : ['0', '30000'],
        'ticklen':20,
        'tick0':0,
        'showline':True,
    },
    'legend':{
        'x':0.45,
        'y':0.04,
        'bordercolor':'#00000',
         'borderwidth':2    
    },
    'width':1200,
    'height':800,
    'margin':go.Margin(
        l=150,
        b=150       ),
    #paper_bgcolor='rgb(233,233,233)',
    #plot_bgcolor='rgb(233,233,233)',
    'font':{'size':32,'family':'arial'},
    

        }


    #title = "UseCase5",
    


fig = {
    'data': data,
    'layout': layout,}


#py.iplot(fig, filename = "4_HyrumReservoir_Curves.py") 


## it can be run from the local machine on Pycharm like this like below
## It would also work here offline but in a seperate window  
#plotly.offline.plot(fig, filename = "4_HyrumReservoir_Curves.py") offline.iplot(fig,filename = 'jupyter/2.2Identify_aggregate_TimeSeriesValues' )       

# offline.iplot(fig,filename = 'jupyter/4_HyrumReservoir_Curves')       

offline.iplot(fig,filename = 'jupyter/4_HyrumReservoir_Curves',
             image='png')
# it might take 30-60 seconds to load the html interactive image 

<a id='section4'></a>
### 4. Creating a new HydroShare resource

The best way to save your data is to put it back into HydroShare and is done using the `createHydroShareResource` function. The first step is to identify the files you want to save to a HydroShare.  The cell below lists all the files in the current working directory.

In [11]:
# Grab all the files in the folder where you are working  
files = !ls
print('My files: %s' % files)

My files: ['2.4_plotcdf.html', 'CDF_data.csv', 'Filtered Data.csv', 'query_resut2.csv', 'query_resut.csv', 'UseCase2.2_dentifyDemandSites_TimeSeriesValues.html', 'WaMDaM_UseCase1.ipynb', 'WaMDaM_Use_Case2.1.ipynb', 'WaMDaM_Use_Case2.2.ipynb', 'WaMDaM_Use_Case2.3.ipynb', 'WaMDaM_Use_Case2.4.ipynb', 'WaMDaM_UseCase3.ipynb', 'WaMDaM_Use_Cases_Overview.ipynb']


In [13]:
# lets save this content as a new resource in HydroShare
abstract = 'This is a demo of the HydroShare Python Notebook Server for WaMDaM'
title = 'WaMDaM_Use_Case2.3'    
keywords = ['Demo', 'JupyterHub']  
rtype = 'genericresource'          

# create a list of files that will be added to the HydroShare resource.
#files = ['WaMDaM_use_cases_Multi_columns.ipynb.ipynb']  # this notebook
        
# create a list of files that will be added to the HydroShare resource.
    
files = [hs.content['BearRiverDatasets_Jan2018.sqlite'],'WaMDaM_Use_Case2.3.ipynb']  


In [ ]:
    
# create the new resource
resource_id = hs.createHydroShareResource(abstract, 
                                          title, 
                                          keywords=keywords, 
                                          resource_type=rtype, 
                                          content_files=files, 
                                          public=True)

Creating HydroShare Resource -